Finetuning eines GPT-3-Modell

In diesem Notebook wird dein Datensatz zusammengestellt, der ausschließlich zum Finetuning eines GPT-3-Modells dient

In [3]:
import pandas as pd
import torch
from transformers import GPT2LMHeadModel, GPT2Tokenizer
import re
import json
import openai
import langdetect

# Datensatz erstellen für GPT3 Finetuning 

In [ ]:
# df_rest = pd.read_csv('01_Raw_Data\dataset_Google-Maps-Reviews-Restaurants_2023-03-26_09-43-08-320.csv')
# df_act = pd.read_csv('01_Raw_Data\dataset_Google-Maps-Reviews-Activities_2023-03-26_11-08-15-435.csv')
# df_hotel = pd.read_csv('01_Raw_Data\dataset_Google-Maps-Reviews-Hotels_2023-03-26_11-34-16-492.csv')


# MIT VIEL MEHR DATEN UND DANN DAFÜR DIE LOCATIONS RAUSSCHMEIßEN

#df_rest = pd.read_csv('01_Raw_Data\dataset_Google-Maps-Reviews-Restaurants_v2_2023-03-30_14-05-51-577.csv')
# df_act = pd.read_csv('01_Raw_Data\dataset_Google-Maps-Reviews-Activities_v2_2023-03-26_11-08-15-435.csv')
# df_hotel = pd.read_csv('01_Raw_Data\dataset_Google-Maps-Reviews-Hotels_v2_2023-03-26_11-34-16-492.csv')

#print('Restaurants (Shape): ', df_rest.shape)
# print('Activities (Shape): ', df_act.shape)
# print('Hotels (Shape): ', df_hotel.shape)

In [ ]:
# drop all nan in imageUrls/0

df_rest = df_rest.dropna(subset=['reviewImageUrls/0'])
print('Restaurants (Shape): ', df_rest.shape)

Idee: nur den Text berücksichtigen. Noch kein Fokus auf zwei Modalitäten

In [ ]:
df_reduced = pd.read_csv('base_keywords_sentiment_reduced.csv')
df_fine_tuning = pd.concat([df_rest, df_act, df_hotel], axis=0)

df_fine_tuning = df_fine_tuning.reset_index(drop=True)
df_fine_tuning = df_fine_tuning.drop_duplicates(subset=['reviewId'])
print('All (raw): ', df_fine_tuning.shape)

df_fine_tuning = df_fine_tuning.dropna(subset=['text'])
df_fine_tuning = df_fine_tuning[['text', 'reviewId', 'url', 'placeId','categoryName', 'stars', 'reviewImageUrls/0']]
print('All with text: ', df_fine_tuning.shape)
df_fine_tuning['text'] = df_fine_tuning['text'].astype(str)

df_fine_tuning = df_fine_tuning[~df_fine_tuning['reviewId'].isin(df_reduced['reviewId'])]
print('All (ohne späteren Base-Datensatz reviews): ', df_fine_tuning.shape)
df_fine_tuning.to_csv('finetuning_gpt3_v2.csv', index=False)

Jetzt. Andere Sprachen rausfiltern und Einträge die im Base-Datensatz sind rausfiltern. @TODO: RAUSFINDEN, OB AUF ANDERE LOCATOINS TRAINIEREN ODER DIE SELBEN WIE IM BASE_..._.csv Datensatz

In [ ]:
df = pd.read_csv ("finetuning_gpt3_v2.csv")
df.shape

In [ ]:
def is_english(text):
    try:
        return langdetect.detect(text) == 'en'
    except:
        return False

print('Starting...', df.shape)
df = df[df['text'].apply(is_english)]

print('Finsihed only English', df.shape)
df.to_csv('finetuning_gpt3_sameplaces_only_english.csv', index=False)

In [25]:
df = pd.read_csv ("finetuning_gpt3_sameplaces_only_english_multimod.csv")
print(df.shape)

df = df.groupby('placeId').head(25)
print(df['placeId'].nunique())

print(df.shape)
df.to_csv('finetuning_gpt3_sameplaces_only_english_multimod_25perplace.csv', index=False)

(5286, 7)
50
(893, 7)


In [31]:
# Rows die END enthlaten rausnehmen, für COMPLETION PROMPT WICHTIG
df = pd.read_csv ("finetuning_gpt3_sameplaces_only_english_multimod_25perplace.csv")
print(df.shape)
df = df[~df['text'].str.contains("END")]
print(df.shape)
df.to_csv('finetuning_gpt3_sameplaces_only_english_multimod_25perplace.csv', index=False)

(890, 7)
(890, 7)


In [43]:
# Checks

df_reduced = pd.read_csv("base_keywords_sentiment_reduced.csv")
df_fine_tuning = pd.read_csv("finetuning_gpt3_sameplaces_only_english_multimod_25perplace.csv")
print(f"Rows that appear in both dataframes:" , df_reduced['reviewId'].isin(df_fine_tuning['reviewId']).sum())

Rows that appear in both dataframes: 0


In [14]:
def generate_prompt_gpt3(row):
    category = row["categoryName"]
    return f"A Google Maps review about a {category}."

# def generate_prompt_gpt35turbo(row):
#     category = row["categoryName"]
#     return f"Write a Google Maps review about a {category}."

# def generate_prompt_gpt35turbo(row):
#     stars = row["stars"]
#     category = row["categoryName"]
#     keywords = row["keywords_only"]
#     keywords = keywords[1:-1].split(",")[:3]
#     keywords = [k.strip() for k in keywords]
#     keywords = ", ".join(keywords)
#     keywords = keywords.replace("'", "")
#     return f"Write a {stars} stars Google Maps review about a {category}, with the following keywords: {keywords}."

In [15]:
df = pd.read_csv ("finetuning_gpt3_sameplaces_only_english_multimod_25perplace.csv")
print(df.shape)

df["prompt"] = df.apply(generate_prompt_gpt3, axis=1)
df["completion"] = df["text"]

print(df.shape)
print(df.columns)
print(df[['completion','prompt']].head(4999))

df.to_csv('finetuning_gpt3_sameplaces_only_english_multimod_25perplace_pc.csv', index=False)

(890, 7)
(890, 9)
Index(['text', 'reviewId', 'url', 'placeId', 'categoryName', 'stars',
       'reviewImageUrls/0', 'prompt', 'completion'],
      dtype='object')
                                            completion  \
0    My husband and I had our anniversary dinner he...   
1    Celebrated my wife’s birthday and she loved it...   
2    Fresh and well seasoned! I like the aesthetic ...   
3    Very beautiful atmosphere and impeccable servi...   
4    The entire staff was attentive and so friendly...   
..                                                 ...   
885  AWESOME!  All employees were great!  The barte...   
886  Beautiful location on the worlds most famous b...   
887  A disappointment. This used to be the premier ...   
888  4 elevators and only 2 in operation\nBed in ou...   
889  Like many have said, the location is perfect, ...   

                                               prompt  
0    A Google Maps review about a Italian restaurant.  
1    A Google Maps review ab